<a href="https://colab.research.google.com/github/alex-petrov-git/dls-homework-sem-2/blob/main/simple_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [ ]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2025-10-05 13:29:33--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.185.43.25, 188.185.45.92, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.43.25|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1199620/files/SO_vectors_200.bin [following]
--2025-10-05 13:29:33--  https://zenodo.org/records/1199620/files/SO_vectors_200.bin
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  31.6MB/s    in 45s     

2025-10-05 13:30:18 (31.0 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [ ]:
! pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 14.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.2
    Uninstalling scipy-1.16.2:
      Successfully uninstalled scipy-1.16.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
openc

In [ ]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [ ]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [ ]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### ***Вопрос 1:***
* Входит ли слово `cat` в топ-5 близких слов к слову `dog`? Какое место оно занимает?


In [ ]:
# method most_similar
wv_embeddings.most_similar(positive=[word], topn=5)

[('animal', 0.8564180135726929),
 ('dogs', 0.7880866527557373),
 ('mammal', 0.7623804211616516),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665)]

***Ваш ответ:*** 'cat' не входит в топ-5 ближайших слов к 'dog'.

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [ ]:
import numpy as np
import re
from transformers import AutoTokenizer
import nltk
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
from nltk.corpus import wordnet
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')

class MyTokenizer:
    def __init__(self):
        self.wordPunctTokenizer = WordPunctTokenizer()
        self.HFtokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
        self.lemmatizer = WordNetLemmatizer()

    def word_punct_tokenize(self, text):
        return self.wordPunctTokenizer.tokenize(text)

    def hf_tokenize(self, text):
        return self.HFtokenizer.tokenize(text)

    def tokenize(self, text):
        return re.findall(r'\w+|\?', text)

    def tokenize_lowercase(self, text):
        tokens = self.tokenize(text)
        return [token.lower() for token in tokens]

    def get_wordnet_pos(self, treebank_tag):
        """Преобразование POS-тегов NLTK в формат WordNet."""
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN  # По умолчанию - существительное

    def lemmatize(self, text):
        tokens = word_tokenize(text)
        pos_tags = pos_tag(tokens)
        lemmas = [self.lemmatizer.lemmatize(token, self.get_wordnet_pos(tag)) for token, tag in pos_tags]
        return lemmas

    def lemmatize_lowercase(self, text):
        text_lower = text.lower()
        tokens = word_tokenize(text_lower)
        pos_tags = pos_tag(tokens)
        lemmas = [self.lemmatizer.lemmatize(token, self.get_wordnet_pos(tag)) for token, tag in pos_tags]
        return lemmas

tokenizer = MyTokenizer()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def question_to_vec(
        question,
        embeddings,
        tokenizer,
        is_lemmatize=False,
        is_lower=False,
        is_word_punct_tokenize=False,
        is_hf_tokenize=False,
        dim=200
    ):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """
    if is_word_punct_tokenize:
        words = tokenizer.word_punct_tokenize(question)
    elif is_hf_tokenize:
        words = tokenizer.hf_tokenize(question)
    elif is_lemmatize and is_lower:
        words = tokenizer.lemmatize_lowercase(question)
    elif is_lemmatize:
        words = tokenizer.lemmatize(question)
    elif is_lower:
        words = tokenizer.tokenize_lowercase(question)
    else:
        words = tokenizer.tokenize(question)

    vec = np.zeros(dim)
    for word in words:
        if word in embeddings:
            vec += embeddings[word]
    if len(words):
        vec /= len(words)

    return vec


Теперь у нас есть метод для создания векторного представления любого предложения.

#### ***Вопрос 2:***

* Какая третья (с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [ ]:
# Предложение
question = "I love neural networks"

round(question_to_vec(question, wv_embeddings, tokenizer)[2], 2)

-0.96

**Ответ:** -0.96

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

Hits@K  измеряет долю вопросов, для которых правильный ответ попал в топ-K позиций среди отранжированных кандидатов.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа.

DCG@K  измеряет качество ранжирования, учитывая не только факт наличия правильного ответа в топ-K, но и ***его точную позицию***.

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1]$

Проверяем условие $ \text{rank}_{q'_1} \leq 1 $: ***условие неверно***.

Следовательно, $[\text{rank}_{q'_1} \leq 1] = 0$.

- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Проверяем условие $ \text{rank}_{q'_1} \leq 4 $: ***условие верно***.

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### ***Вопрос 3***:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)



$\text{DCG@10} = \frac{1}{\log_2(1+9)}\cdot[9 \le 10] = \frac{1}{\log_2{10}} = 0.3$

In [ ]:
1/np.log2(10)

0.3010299956639812

#### Более сложный пример оценок

Рассмотрим пример с $ N > 1 $, где $ N = 3 $ (три вопроса) и для каждого вопроса заданы позиции их дубликатов. Вычислим метрики **Hits@K** для разных значений $ K $.

---

- $ N = 3 $: Три вопроса ($ q_1, q_2, q_3 $).
- Для каждого вопроса известна позиция его дубликата ($ \text{rank}_{q'_i} $):
  - $ \text{rank}_{q'_1} = 2 $,
  - $ \text{rank}_{q'_2} = 5 $,
  - $ \text{rank}_{q'_3} = 1 $.

Мы будем вычислять **Hits@K** для $ K = 1, 5 $.

---

**Для $ K = 1 $:**

Подставим значения:
$$
\text{Hits@1} = \frac{1}{3} \cdot \left( [\text{rank}_{q'_1} \leq 1] + [\text{rank}_{q'_2} \leq 1] + [\text{rank}_{q'_3} \leq 1] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 1 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 1 $ → $ 1 $.

Сумма:
$$
\text{Hits@1} = \frac{1}{3} \cdot (0 + 0 + 1) = \frac{1}{3}.
$$

$$
\boxed{\text{Hits@1} = \frac{1}{3}}.
$$

---

**Для $ K = 5 $:**

Подставим значения:
$$
\text{Hits@5} = \frac{1}{3} \cdot \left( [\text{rank}_{q'_1} \leq 5] + [\text{rank}_{q'_2} \leq 5] + [\text{rank}_{q'_3} \leq 5] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 5 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 5 $ → $ 1 $.

Сумма:
$$
\text{Hits@5} = \frac{1}{3} \cdot (1 + 1 + 1) = 1.
$$

$$
\boxed{\text{Hits@5} = 1}.
$$

Теперь вычислим метрику **DCG@K** для того же примера, где $ N = 3 $ (три вопроса), и для каждого вопроса известна позиция его дубликата ($ \text{rank}_{q'_i} $):

- $ \text{rank}_{q'_1} = 2 $,
- $ \text{rank}_{q'_2} = 5 $,
- $ \text{rank}_{q'_3} = 1 $.

Мы будем вычислять **DCG@K** для $ K = 1, 5 $.

---
**Для $ K = 1 $:**
Подставим значения:
$$
\text{DCG@1} = \frac{1}{3} \cdot \left( \frac{1}{\log_2(1 + \text{rank}_{q'_1})} \cdot [\text{rank}_{q'_1} \leq 1] + \frac{1}{\log_2(1 + \text{rank}_{q'_2})} \cdot [\text{rank}_{q'_2} \leq 1] + \frac{1}{\log_2(1 + \text{rank}_{q'_3})} \cdot [\text{rank}_{q'_3} \leq 1] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 1 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 1 $ → $ 1 $.

Сумма:
$$
\text{DCG@1} = \frac{1}{3} \cdot (0 + 0 + 1) = \frac{1}{3}.
$$
$$
\boxed{\text{DCG@1} = \frac{1}{3}}.
$$

---


**Для $ K = 5 $:**
Подставим значения:
$$
\text{DCG@5} = \frac{1}{3} \cdot \left( \frac{1}{\log_2(1 + \text{rank}_{q'_1})} \cdot [\text{rank}_{q'_1} \leq 5] + \frac{1}{\log_2(1 + \text{rank}_{q'_2})} \cdot [\text{rank}_{q'_2} \leq 5] + \frac{1}{\log_2(1 + \text{rank}_{q'_3})} \cdot [\text{rank}_{q'_3} \leq 5] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 5 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 5 $ → $ 1 $.

Сумма:
$$
\text{DCG@5} = \frac{1}{3} \cdot (0.631 + 0.387 + 1) = \frac{1}{3} \cdot 2.018 \approx 0.673.
$$

$$
\boxed{\text{DCG@5} \approx 0.673}.
$$

#### ***Вопрос 4:***
* Найдите максимум `Hits@47 - DCG@1`?



$$\text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K]$$

$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$

$$\underset{q_i}{\text{max }} \text{(Hits@47 $-$ DCG@1)} = \dfrac{1}{3}\left(1+1+1 - 0 - 0 - 0\right)$$

Достигается, если $1 < \text{rank}_{q'_i} < 48$

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$.

$dup\_ranks$ является списком, который содержит рейтинги дубликатов (их позиции в ранжированном списке).

К примеру для <font color='red'>"Что такое язык python?"</font> $dup\_ranks = [2]$.

In [ ]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        k: пороговое значение для ранга
        result: вернуть Hits@k
    """
    # Подсчитываем количество дубликатов, чей ранг <= k
    hits_value = sum(int(rank <= k) for rank in dup_ranks)/len(dup_ranks)
    return hits_value

In [ ]:
dup_ranks = [2]

k = 1
hits_value = hits_count(dup_ranks, k)
print(f"Hits@1 = {hits_value}")

k = 4
hits_value = hits_count(dup_ranks, k)
print(f"Hits@4 = {hits_value}")

Hits@1 = 0.0
Hits@4 = 1.0


In [ ]:
import math

def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        k: пороговое значение для ранга
        result: вернуть DCG@k
    """
    dcg_value = sum(int(rank <= k)/math.log2(1+rank) for rank in dup_ranks)/len(dup_ranks)
    return dcg_value

In [ ]:
# Пример списка позиций дубликатов
dup_ranks = [2]

# Вычисляем DCG@1
dcg_value = dcg_score(dup_ranks, k=1)
print(f"DCG@1 = {dcg_value:.3f}")

# Вычисляем DCG@4
dcg_value = dcg_score(dup_ranks, k=4)
print(f"DCG@4 = {dcg_value:.3f}")

DCG@1 = 0.000
DCG@4 = 0.631


Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [ ]:
import pandas as pd

In [ ]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наши кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]

# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [ ]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [ ]:
!unzip stackoverflow_similar_questions.zip

Archive:  stackoverflow_similar_questions.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


Считайте данные.

In [ ]:
def read_corpus(filename):
    data = []
    with open(filename, encoding='utf-8') as file:
        for line in file:
            data.append(line.strip().split('\t'))
    return data

Нам понадобиться только файл validation.

In [ ]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [ ]:
len(validation_data)

3760

Размер нескольких первых строк

In [ ]:
for i in range(25):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001
6 1001
7 1001
8 1001
9 1001
10 1001
11 1001
12 1001
13 1001
14 1001
15 1001
16 1001
17 1001
18 1001
19 1001
20 1001
21 1001
22 1001
23 1001
24 1001
25 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b)/np.sqrt(np.dot(a, a)*np.dot(b, b) + 1e-8)

In [ ]:
def rank_candidates(
        question,
        candidates,
        embeddings,
        tokenizer,
        is_lemmatize=False,
        is_lower=False,
        is_word_punct_tokenize=False,
        is_hf_tokenize=False,
        dim=200
    ):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    q_vec = question_to_vec(
        question,
        embeddings,
        tokenizer,
        is_lemmatize=is_lemmatize,
        is_lower=is_lower,
        is_word_punct_tokenize=is_word_punct_tokenize,
        is_hf_tokenize=is_hf_tokenize,
    )

    result = []

    for i, candidate in enumerate(candidates):
        c_vec = question_to_vec(
            candidate,
            embeddings,
            tokenizer,
            is_lemmatize=is_lemmatize,
            is_word_punct_tokenize=is_word_punct_tokenize,
            is_hf_tokenize=is_hf_tokenize,
        )
        similarity = cosine_similarity(q_vec, c_vec)
        result.append(((i, candidate), similarity))

    result = sorted(
        result,
        key=lambda x: x[1],
        reverse=True
    )

    return [item[0] for item in result]

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [ ]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [ ]:
for question, q_candidates in zip(questions, candidates):
    ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
    ranks_lemmatized = rank_candidates(question, q_candidates, wv_embeddings, tokenizer, is_lemmatize=True)
    ranks_lower = rank_candidates(question, q_candidates, wv_embeddings, tokenizer, is_lower=True)
    ranks_lemmatized_lower = rank_candidates(question, q_candidates, wv_embeddings, tokenizer, is_lemmatize=True, is_lower=True)
    ranks_word_punct = rank_candidates(question, q_candidates, wv_embeddings, tokenizer, is_word_punct_tokenize=True)
    ranks_hf = rank_candidates(question, q_candidates, wv_embeddings, tokenizer, is_hf_tokenize=True)

    print("ranks: ", ranks)
    print("ranks_lemmatized: ", ranks_lemmatized)
    print("ranks_lower: ", ranks_lower)
    print("ranks_lemmatized_lower: ", ranks_lemmatized_lower)
    print("ranks_word_punct: ", ranks_word_punct)
    print("ranks_hf: ", ranks_hf)

    print()

ranks:  [(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]
ranks_lemmatized:  [(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]
ranks_lower:  [(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]
ranks_lemmatized_lower:  [(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]
ranks_word_punct:  [(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]
ranks_hf:  [(0, 'Convert Google results object (pure js) to Python obj

Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### ***Вопрос 5:***
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

Ответ: 102

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
wv_ranking = []
wv_ranking_low = []
wv_ranking_hf = []
wv_ranking_word_punkt = []
wv_ranking_lem = []
wv_ranking_lem_low = []

max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line

    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    ranks_low = rank_candidates(q, ex, wv_embeddings, tokenizer, is_lower = True)
    wv_ranking_low.append([r[0] for r in ranks_low].index(0) + 1)
    ranks_hf = rank_candidates(q, ex, wv_embeddings, tokenizer, is_hf_tokenize=True)
    wv_ranking_hf.append([r[0] for r in ranks_hf].index(0) + 1)
    ranks_word_punkt = rank_candidates(q, ex, wv_embeddings, tokenizer, is_word_punct_tokenize=True)
    wv_ranking_word_punkt.append([r[0] for r in ranks_word_punkt].index(0) + 1)
    ranks_lem = rank_candidates(q, ex, wv_embeddings, tokenizer, is_lemmatize=True)
    wv_ranking_lem.append([r[0] for r in ranks_lem].index(0) + 1)
    ranks_lem_low = rank_candidates(q, ex, wv_embeddings, tokenizer, is_lemmatize=True, is_lower=True)
    wv_ranking_lem_low.append([r[0] for r in ranks_lem_low].index(0) + 1)


  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
from tabulate import tabulate

# Список токенизаторов и их результатов
tokenizers = [
    ("Default", wv_ranking),
    ("Lowercase", wv_ranking_low),
    ("HF Tokenizer", wv_ranking_hf),
    ("Word Punct", wv_ranking_word_punkt),
    ("Lemmatize", wv_ranking_lem),
    ("Lemmatize Lower", wv_ranking_lem_low)
]

# Значения k
ks = [1, 5, 10, 100, 250, 500, 1000]

# Подготовка данных для таблицы DCG
dcg_table_data = []
dcg_headers = ["Tokenizer"] + [f"DCG@{k}" for k in ks]

for name, ranking in tokenizers:
    row = [name]
    for k in ks:
        row.append(f"{dcg_score(ranking, k):.3f}")
    dcg_table_data.append(row)

# Вывод таблицы DCG
print("DCG Scores:")
print(tabulate(dcg_table_data, headers=dcg_headers, tablefmt="grid"))
print()

# Подготовка данных для таблицы Hits
hits_table_data = []
hits_headers = ["Tokenizer"] + [f"Hits@{k}" for k in ks]

for name, ranking in tokenizers:
    row = [name]
    for k in ks:
        row.append(f"{hits_count(ranking, k):.3f}")
    hits_table_data.append(row)

# Вывод таблицы Hits
print("Hits Counts:")
print(tabulate(hits_table_data, headers=hits_headers, tablefmt="grid"))

DCG Scores:
+-----------------+---------+---------+----------+-----------+-----------+-----------+------------+
| Tokenizer       |   DCG@1 |   DCG@5 |   DCG@10 |   DCG@100 |   DCG@250 |   DCG@500 |   DCG@1000 |
+=================+=========+=========+==========+===========+===========+===========+============+
| Default         |   0.285 |   0.342 |    0.36  |     0.406 |     0.42  |     0.431 |      0.444 |
+-----------------+---------+---------+----------+-----------+-----------+-----------+------------+
| Lowercase       |   0.258 |   0.333 |    0.355 |     0.402 |     0.417 |     0.425 |      0.438 |
+-----------------+---------+---------+----------+-----------+-----------+-----------+------------+
| HF Tokenizer    |   0.361 |   0.435 |    0.456 |     0.501 |     0.514 |     0.522 |      0.528 |
+-----------------+---------+---------+----------+-----------+-----------+-----------+------------+
| Word Punct      |   0.276 |   0.332 |    0.349 |     0.395 |     0.41  |     0.421 |  

Из формул выше можно понять, что

- $ \text{Hits@K} $ **монотонно неубывающая функция** $ K $, которая стремится к 1 при $ K \to \infty $.

- $ \text{DCG@K} $ **монотонно неубывающая функция** $ K $, но рост замедляется с увеличением $ K $ из-за убывания веса $ \frac{1}{\log_2(1 + \text{rank}_{q'_i})} $.

### Эмбеддинги, обученные на корпусе похожих вопросов

In [ ]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

***Рассмотрим подробнее*** данное склеивание.

1. Каждая строка из train_data разбивается на вопрос (question) и список кандидатов.

2. Для каждого кандидата вопрос склеивается с ним в одну строку.

3. Склеенная строка (combined_text) токенизируется, и полученный список токенов добавляется в общий корпус (corpus).

***Пример***

    Вопрос: "What is Python?"
    Кандидаты: ["Python is a programming language", "Java is another language"]
    Склеенные строки:
        "What is Python? Python is a programming language"
        "What is Python? Java is another language"
         
    Токенизированные списки:
        ['what', 'is', 'python', 'python', 'is', 'a', 'programming', 'language']
        ['what', 'is', 'python', 'java', 'is', 'another', 'language']
         
     

In [ ]:
train_data[111258]

['Determine if the device is a smartphone or tablet?',
 'Change imageView params in all cards together']

In [ ]:
# Создаем общий корпус текстов
corpus = []

for pair in train_data:
    corpus.append(tokenizer.tokenize(' '.join(pair)))

In [ ]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(
    sentences=corpus,
    vector_size=200,
    window=5,
    min_count=3,
    workers=4,
    sg=1,
#    epochs=10,
#    negative=15,
#    compute_loss=True
).wv

In [ ]:
wv_ranking = []
wv_ranking_low = []
wv_ranking_hf = []
wv_ranking_word_punkt = []
wv_ranking_lem = []
wv_ranking_lem_low = []

max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line

    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    ranks_low = rank_candidates(q, ex, embeddings_trained, tokenizer, is_lower=True)
    wv_ranking_low.append([r[0] for r in ranks_low].index(0) + 1)
    ranks_hf = rank_candidates(q, ex, embeddings_trained, tokenizer, is_hf_tokenize=True)
    wv_ranking_hf.append([r[0] for r in ranks_hf].index(0) + 1)
    ranks_word_punkt = rank_candidates(q, ex, embeddings_trained, tokenizer, is_word_punct_tokenize=True)
    wv_ranking_word_punkt.append([r[0] for r in ranks_word_punkt].index(0) + 1)
    ranks_lem = rank_candidates(q, ex, embeddings_trained, tokenizer, is_lemmatize=True)
    wv_ranking_lem.append([r[0] for r in ranks_lem].index(0) + 1)
    ranks_lem_low = rank_candidates(q, ex, embeddings_trained, tokenizer, is_lemmatize=True, is_lower=True)
    wv_ranking_lem_low.append([r[0] for r in ranks_lem_low].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
tokenizers = [
    ("Default", wv_ranking),
    ("Lowercase", wv_ranking_low),
    ("HF Tokenizer", wv_ranking_hf),
    ("Word Punct", wv_ranking_word_punkt),
    ("Lemmatize", wv_ranking_lem),
    ("Lemmatize Lower", wv_ranking_lem_low)
]

ks = [1, 5, 10, 100, 250, 500, 1000]

dcg_table_data = []
dcg_headers = ["Tokenizer"] + [f"DCG@{k}" for k in ks]

for name, ranking in tokenizers:
    row = [name]
    for k in ks:
        row.append(f"{dcg_score(ranking, k):.3f}")
    dcg_table_data.append(row)

print("DCG Scores:")
print(tabulate(dcg_table_data, headers=dcg_headers, tablefmt="grid"))
print()

hits_table_data = []
hits_headers = ["Tokenizer"] + [f"Hits@{k}" for k in ks]

for name, ranking in tokenizers:
    row = [name]
    for k in ks:
        row.append(f"{hits_count(ranking, k):.3f}")
    hits_table_data.append(row)

print("Hits Counts:")
print(tabulate(hits_table_data, headers=hits_headers, tablefmt="grid"))

DCG Scores:
+-----------------+---------+---------+----------+-----------+-----------+-----------+------------+
| Tokenizer       |   DCG@1 |   DCG@5 |   DCG@10 |   DCG@100 |   DCG@250 |   DCG@500 |   DCG@1000 |
+=================+=========+=========+==========+===========+===========+===========+============+
| Default         |   0.376 |   0.459 |    0.48  |     0.521 |     0.533 |     0.541 |      0.546 |
+-----------------+---------+---------+----------+-----------+-----------+-----------+------------+
| Lowercase       |   0.329 |   0.406 |    0.429 |     0.474 |     0.487 |     0.495 |      0.503 |
+-----------------+---------+---------+----------+-----------+-----------+-----------+------------+
| HF Tokenizer    |   0.353 |   0.424 |    0.441 |     0.478 |     0.494 |     0.504 |      0.511 |
+-----------------+---------+---------+----------+-----------+-----------+-----------+------------+
| Word Punct      |   0.377 |   0.459 |    0.48  |     0.521 |     0.533 |     0.541 |  

### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


Получилась нелинейная зависимость качества от способов токенизации и обучения эмбеддингов. Как видно, в первом случае (для скачанных эмбеддингов) лучшим токенайзером оказался hf_tokenizer (который производит lowercase, разбиение слов на основу, суффиксы и префиксы), а во втором - дефолтный токенайзер.

Не знаю каким образом обучались первые эмбеддинги, но генерально оказывается (для всех токенайзеров), что обучение эмбеддингов на парах вопрос-ответ лучше, чем в первом случае, так как эмбеддинги при обучении запоминают более релевантный контекст для каждого слова для решения задачи ранжирования ответов на вопросы.